In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#Urllib.request for opening url and Beautiful soup for extracting html data
from urllib.request import urlopen as uReq
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#### Data Scrapping

In [3]:
content = requests.get(url)

#html_content = content.text

In [4]:
soup = BeautifulSoup(content.content, "lxml")


In [5]:
My_table = soup.find('table',{'class':'wikitable sortable'}).tbody
#My_table

In [6]:
rows = My_table.find_all('tr')

In [7]:
columns = [v.text.replace('\n', '') for v in rows[0].find_all('th')]

In [9]:
df = pd.DataFrame(columns=columns)

In [10]:
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', '')]

#Pandas

    df = df.append(pd.Series(values, index=columns), ignore_index=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [11]:
df = df[df['Borough'] != 'Not assigned']

df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [12]:
#Shape of the table
df.shape

(211, 3)

In [24]:
Neigh = df.groupby(['Postcode','Borough'], sort = False).agg(lambda x: ', '.join(x))

In [30]:
Neigh.reset_index(inplace=True)

In [31]:
Neigh.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [25]:
df1 = pd.read_csv('Geospatial_Coordinates.csv')

### Geospatial data

In [37]:
dfg = df1.rename(columns={'Postal Code':'Postcode'})

dfg.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [32]:
#Merge both dataframe
Neighbourhood = Neigh.merge(dfg, on='Postcode', how='left')

In [33]:
Neighbourhood.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494


In [35]:
Neighbourhood['Neighbourhood']=np.where(Neighbourhood['Neighbourhood'] == 'Not assigned', Neighbourhood['Borough'], Neighbourhood['Neighbourhood'])
Neighbourhood.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [36]:
Neighbourhood.shape

(103, 5)